In [107]:
# Dependencies and set-up
import pandas as pd
import requests
import json
from config import api_key
from pprint import pprint
from datetime import date
from matplotlib import pyplot as plt
import numpy as np
import gmaps
from config import g_key
import scipy.stats as stats
from datetime import date